In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [2]:
%tensorflow_version 1.x

import cv2
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import plot_model

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16, InceptionResNetV2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [3]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [4]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [6]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state = 21)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_df

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
7376,resized/7377.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
4717,resized/4718.jpg,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
9356,resized/9357.jpg,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0
5826,resized/5827.jpg,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0
10038,resized/10039.jpg,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9336,resized/9337.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
48,resized/49.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1
8964,resized/8965.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
5944,resized/5945.jpg,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=16,augmentation=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,))
valid_gen = PersonDataGenerator(val_df, batch_size=16,augmentation=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,),shuffle=False)

In [0]:
# val_gen = PersonDataGenerator(val_df, batch_size=64)

In [8]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
# backbone = VGG16(
#     weights="imagenet", 
#     include_top=False, 
#     input_tensor=Input(shape=(224, 224, 3))
# )
back = InceptionResNetV2(include_top=False,weights=None, input_tensor=Input(shape=(224, 224, 3)))

neck = back.output
neck = BatchNormalization()(neck)
neck = Dense(512, activation="relu")(neck)
neck = Flatten(name="flatten")(neck)
neck = Dense(8256, activation="relu")(neck)
neck = BatchNormalization()(neck)


def build_tower(in_layer):
    neck = Dropout(0.15)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.2)(in_layer)
    neck = Dense(8, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# def build_tower_age(in_layer):
#     neck = Dropout(0.2)(in_layer)
#     neck = Dense(512, activation="relu")(neck)
#     neck = Dropout(0.2)(in_layer)
#     neck = Dense(512, activation="relu")(neck)
#     neck = Dropout(0.2)(in_layer)
#     neck = Dense(256, activation="relu")(neck)
#     neck = Dropout(0.2)(in_layer)
#     neck = Dense(512, activation="relu")(neck)
#     neck = Dropout(0.2)(in_layer)
#     neck = Dense(128, activation="relu")(neck)
#     return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)


# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=back.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
# plot_model(model, to_file='/content/gdrive/My Drive/model.png')


In [0]:
model.summary()

In [0]:
# # freeze backbone
# for layer in backbone.layers:
# 	layer.trainable = False

In [11]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [12]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=1, 
    epochs=20,
    verbose=1
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
721/721 [==============================] - 292s 405ms/step - loss: 8.8697 - gender_output_loss: 0.7048 - image_quality_output_loss: 1.1741 - age_output_loss: 1.5395 - weight_output_loss: 1.0865 - bag_output_loss: 0.9727 - footwear_output_loss: 1.2130 - pose_output_loss: 1.0313 - emotion_output_loss: 1.1479 - gender_output_acc: 0.5773 - image_quality_output_acc: 0.5075 - age_output_acc: 0.3604 - weight_output_acc: 0.6108 - bag_output_acc: 0.5402 - footwear_output_acc: 0.5123 - pose_output_acc: 0.5863 - emotion_output_acc: 0.6709 - val_loss: 7.8001 - val_gender_output_loss: 0.6692 - val_image_quality_output_loss: 0.9888 - val_age_output_loss: 1.4345 - val_weight_output_loss: 0.9367 - val_bag_output_loss: 0.9561 - val_footwear_output_loss: 0.9544 - val_pose_output_loss: 0.8922 - val_emotion_output_loss: 0.9682 - val_gender_output_acc: 0.5994 - val_image_quality_output_acc: 0.5600 - v

In [13]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=1, 
    epochs=10,
    verbose=1
)

Epoch 1/10
721/721 [==============================] - 227s 315ms/step - loss: 1.5971 - gender_output_loss: 0.0830 - image_quality_output_loss: 0.2343 - age_output_loss: 0.3339 - weight_output_loss: 0.2361 - bag_output_loss: 0.1706 - footwear_output_loss: 0.1930 - pose_output_loss: 0.1019 - emotion_output_loss: 0.2443 - gender_output_acc: 0.9681 - image_quality_output_acc: 0.9096 - age_output_acc: 0.8774 - weight_output_acc: 0.9081 - bag_output_acc: 0.9346 - footwear_output_acc: 0.9297 - pose_output_acc: 0.9659 - emotion_output_acc: 0.9069 - val_loss: 14.1522 - val_gender_output_loss: 0.8522 - val_image_quality_output_loss: 2.0260 - val_age_output_loss: 2.7820 - val_weight_output_loss: 1.9231 - val_bag_output_loss: 1.7897 - val_footwear_output_loss: 1.6470 - val_pose_output_loss: 1.1335 - val_emotion_output_loss: 1.9986 - val_gender_output_acc: 0.7697 - val_image_quality_output_acc: 0.4577 - val_age_output_acc: 0.3263 - val_weight_output_acc: 0.5792 - val_bag_output_acc: 0.5881 - val_fo

In [1]:
model.save('/content/gdrive/My Drive/eip/30epochRes.h5')

NameError: ignored

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=1, 
    epochs=20,
    verbose=1
)

In [0]:
model.save('/content/gdrive/My Drive/eip/50epochRes.h5')

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=1, 
    epochs=20,
    verbose=1
)

In [0]:
model.save('/content/gdrive/My Drive/eip/70epochRes.h5')

In [0]:
results = model.evaluate_generator(valid_gen,verbose=1)
dict(zip(model.metrics_names, results))

In [0]:
model.save('/content/gdrive/My Drive/eip/30epoch.h5')

In [0]:
# def softmax(scores):
#     exp = np.exp(scores)
#     scores = exp / np.sum(exp)
#     return scores
# softmax([5, 7, 4, 6])


In [0]:
# score = model.evaluate(X_test, Y_test, verbose=0)

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1
)

In [0]:
model.summary()

In [0]:
model.history

In [0]:
test_acc = model.evaluate_generator(val_gen, steps=50)
print('test acc:', test_acc)


In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()